# 0. 전에 쓰던 함수, 모델 불러오기

In [50]:
# 필요한 패키지 import
import torch
import numpy as np
from tqdm import tqdm

from transformers import BertTokenizer, BertModel, AdamW
from transformers import BertForMaskedLM,BertForSequenceClassification

import torch.nn.functional as F
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import random
import time
import datetime

In [26]:
# 파일 읽기
def read_file(filename):
    with open(filename, "r") as file:
        lines = file.readlines()
        l = []
        for line in lines:
            l.append(line.strip().split("</B>")[1])
#             print(line)
    return l
verb_hao = read_file("../data/verb_hao.txt")
verb_dao = read_file("../data/verb_dao.txt")
verb_xialai = read_file("../data/verb_xialai.txt")
verb_xiaqu = read_file("../data/verb_xiaqu.txt")

In [27]:
!head ../data/verb_hao.txt

﻿<B> 科技文献</B>是约４２ｍｍ的遮光罩镜头外型尺寸为以说一尘不染，可见其密封性<U>恨好</U>。如此笔者发现独立镜头厂家适配０７２５ｘ１２２ｍｍ光学构造为１
<B> 科技文献</B>很爱听，真像换了个脑筋，明年养多少头牛、羊都一家一户地帮我们<U>算好</U>了。”东湾乡大泉村二组村民马建雄忙着拉干部到自家去住，“干部住
<B> 科技文献</B>有关重大问题。第三，企业档案处置具体工作是收集、整理、统计、<U>保管好</U>企业已经形成的档案，清点库存，按有关规定做好档案留存与销毁的鉴
<B> 科技文献</B>策与财政政策的协调配合还体现为在经济发展的不同时期，两者都要<U>搭配好</U>。１９９８年下半年，财政为扩大内需，向国有商业银行增加发行１０
<B> 科技文献</B>政府贯彻〈中国教育改革和发展纲要〉的意见》。发展教育事业注意<U>处理好</U>三个关系：在数量和质量的关系上，更加重视质量；在德育和智育的关
<B> 科技文献</B>学管理的示范。一是要通过学习，增强创新创业的自觉性。要创业、<U>干好</U>事业，就必须抓重点、带全局。所有的人力、智力、财力、物力、科技
<B> 科技文献</B>解放思想，寻找经济快速发展的新路子。只有人的思想解放了，才能<U>运用好</U>中央制定的各项方针和政策，才能敢“闯”敢“新”，走出一条独特的
<B> 科技文献</B>到手术室，手术室大空间，手术床小范围，手术室到ＩＣＵ等各环节<U>做好</U>保温，避免外介温度的大波动引起婴儿不良反应。３．３减少患婴术中
<B> 科技文献</B>例如，在学习“如何做小主人和小客人”这一社会知识时，我们事先<U>联系好</U>一个家庭，并向主人详细讲解此次活动的目的、方法、步骤，以取得默
<B> 科技文献</B>切实加强农村信用社金融监管与行业管理的初步设想（一）必须正确<U>处理好</U>三大关系回、正确处理好信用社与联社的关系。基本原则是既要尊重信


In [29]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 2 GPU(s) available.
We will use the GPU: TITAN RTX


In [51]:
# 모델, 배치, 최대 길이 정의
MODEL_TYPE = 'bert-base-chinese'
MAX_SIZE = 150
BATCH_SIZE = 200

In [52]:
# 정확도 구하는 함수 통합
def get_accuracy(data, top_n = 1):
    def preprocessing(sentences) :
        sent = []
        label = []
        for sentence in sentences : 
            s1 = sentence.split("<U>")
            s2 = s1[1].split("</U>")
            label.append(s2[0][-1])
            sent.append(s1[0]+s2[0][:-1]+"[MASK]"+s2[1])
    #         print(s2[0][-1])
    #         print(s1[0]+s2[0][:-1]+"[MASK]"+s2[1])
        return sent, label
    def inference(sentences):
        predicts = []
        for sentence in sentences:
            input_ids = tokenizer(sentence, return_tensors="pt")["input_ids"]
            mask_idx = input_ids.tolist()[0].index(103) #103 is [MASK] id
            outputs = model(input_ids)
            logits = outputs[0].detach().numpy()
            pred = tokenizer.convert_ids_to_tokens(np.argmax(logits, axis=2)[0])
            predicts.append(pred[mask_idx])
        return predicts
    
    def softmax_inference(sentences, top_n=5):
        predicts = []
        probabilities = []
        for sentence in sentences:
            input_ids = tokenizer(sentence, return_tensors="pt")["input_ids"]
            mask_idx = input_ids.tolist()[0].index(103) #103 is [MASK] id
            outputs = model(input_ids)
            logits = outputs[0].detach()
            mask_predict = F.softmax(logits[0][mask_idx], dim=0).numpy()
            predict_top_n = mask_predict.argsort()[-top_n:][::-1]
            predicts.append(tokenizer.convert_ids_to_tokens(predict_top_n))
            probabilities.append(mask_predict[predict_top_n].tolist())
        return predicts, probabilities

    def evaluate(pred, label):
        cnt = 0
        for p, l in zip(pred, label):
            if p==l : 
                cnt += 1
        return cnt / len(pred)
    
    if top_n > 1:
        sent, label = preprocessing(data)
        pred, prob = softmax_inference(sent, top_n)
        for sentence, candidates, probabilities, label in zip(sent, pred, prob, label):
            print(sentence)
            for c, p in zip(candidates,  probabilities):
                print("{} : {:.2f}%".format(c, p*100))
            print("Answer : {}".format(label))
            print("")
    sent, label = preprocessing(data)
    predict = inference(sent)
    return evaluate(predict, label)

# 1. Language model fine-tune

## 1.1 데이터를 raw text로 변환

In [53]:
verb_hao[0]

'是约４２ｍｍ的遮光罩镜头外型尺寸为以说一尘不染，可见其密封性<U>恨好</U>。如此笔者发现独立镜头厂家适配０７２５ｘ１２２ｍｍ光学构造为１'

In [70]:
line = verb_hao[0]
line.replace('<U>',"").replace('</U>',"")

'是约４２ｍｍ的遮光罩镜头外型尺寸为以说一尘不染，可见其密封性恨好。如此笔者发现独立镜头厂家适配０７２５ｘ１２２ｍｍ光学构造为１'

In [79]:
def save_as_raw_text(data, filename):
    with open(filename, 'w') as f:
        for d in data:
            for line in d:
                line = line.replace('<U>',"").replace('</U>',"")
                f.write(line)
verb_hao_dao = [verb_hao, verb_dao]
save_as_raw_text(verb_hao_dao, "../data/verb_raw.txt")

In [98]:
# 만들어진 verb hao와 verb dao의 raw text를 사용하여 기존의 pretrained BERT(bert-base-chinese) 모델에 추가로 더 학습시켰다. 그 다음에 아래에서 새롭게 모델을 load함

In [93]:
tokenizer = BertTokenizer.from_pretrained('../src/models/hao_dao')
model = BertForMaskedLM.from_pretrained('../src/models/hao_dao')

In [94]:
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

### previous model accuracies
verb hao : 0.6897
verb dao : 0.7474

### pretrianed model accuracies

In [95]:
verb_hao_acc = get_accuracy(verb_hao)
verb_dao_acc = get_accuracy(verb_dao)

In [96]:
print("Verb_hao accuracy : ",verb_hao_acc)
print("Verb_dao accuracy : ",verb_dao_acc)

Verb_hao accuracy :  0.915
Verb_dao accuracy :  0.9416


In [99]:
# 성능이 큰 폭으로 오르는 것으로 보아, 이 방법이 매우 효과가 있어보임.

# 2. 2음절 Tokenizing

In [101]:
tokenizer.add_tokens(['下来','下去'])

2

In [113]:
def predict():
    sent = input('Input Sentence')
    input_ids = tokenizer(sent, return_tensors="pt")["input_ids"]
    mask_idx = input_ids.tolist()[0].index(103) #103 is [MASK] id
    model.eval()
    outputs = model(input_ids)
    logits = outputs[0].detach()
    mask_predict = F.softmax(logits[0][mask_idx], dim=0).numpy()
    predict_top_n = mask_predict.argsort()[-5:][::-1]
    pred = list()
    prob = list()
    pred.append(tokenizer.convert_ids_to_tokens(predict_top_n))
    prob.append(mask_predict[predict_top_n].tolist())
    for sentence, candidates, probabilities in zip(sent, pred, prob) :
        print(sentence)
        for c, p in zip(candidates,  probabilities):
            print("{} : {:.2f}%".format(c, p*100))
        print("")

In [114]:
predict()

Input Sentence这几天天气突然冷[MASK]了
这
了 : 21.12%
透 : 10.32%
些 : 7.15%
下 : 4.80%
起 : 4.06%

